In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf

train_data = np.load('/content/drive/MyDrive/HW12/train_data.npy')
train_label = np.load('/content/drive/MyDrive/HW12/train_label.npy')
test_data = np.load('/content/drive/MyDrive/HW12/test_data.npy')

In [3]:
# Shuffle the training data
n_train = tf.shape(train_data)[0]
n_test = tf.shape(test_data)[0]

zip_train = list(zip(train_data, train_label))
np.random.shuffle(zip_train)
train_data, train_label = zip(*zip_train)

train_data = tf.reshape(train_data, (5000, 28, 28, 1))

In [4]:
print("Number of training set: "+ str(n_train))
print("Number of testing set: "+ str(n_test))

Number of training set: tf.Tensor(5000, shape=(), dtype=int32)
Number of testing set: tf.Tensor(100000, shape=(), dtype=int32)


In [5]:
# Define optimizers for feature extractor, classifier and domain discriminator
extractor_optimizer = tf.keras.optimizers.Adam(1e-4)
c_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)

In [6]:
# Define feature extractor
extractor = tf.keras.Sequential([
                                 tf.keras.Input(shape=(28, 28, 1)),
                                 tf.keras.layers.Conv2D(64, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Conv2D(128, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Conv2D(256, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Conv2D(512, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Flatten(),

                                 tf.keras.layers.Dense(512)
                                 
])
x = tf.random.normal((10, 28, 28, 1))
print(tf.shape(extractor(x)))

tf.Tensor([ 10 512], shape=(2,), dtype=int32)


In [7]:
# Define domain classifier
domain_classifier = tf.keras.Sequential([
                                     tf.keras.Input(shape=(512, )),
                                     tf.keras.layers.Dense(128, activation='relu'),
                                     tf.keras.layers.Dense(64, activation='relu'),
                                     tf.keras.layers.Dense(1)
])

In [8]:
# Define label classifier
label_classifier = tf.keras.Sequential([
                                        tf.keras.Input(shape=(512, )),
                                        tf.keras.layers.Dense(128, activation='relu'),
                                        tf.keras.layers.Dense(64, activation='relu'),
                                        tf.keras.layers.Dense(10, activation='softmax')
])

In [9]:
# Define losses for label classifier
def classifier_loss(source_label, source_label_output):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy()
  loss = cross_entropy(source_label, source_label_output)
  return loss

# Define losses for domain classifier (discriminator)
def d_loss(source_domain, target_domain):
  cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  target_loss = cross_entropy(tf.ones_like(target_domain), target_domain)
  source_loss = cross_entropy(tf.zeros_like(source_domain), source_domain)
  return target_loss + source_loss

# Define losses for feature extractor
def extractor_loss(source_feature, target_feature, source_label):
  source_label_output = label_classifier(source_feature)
  source_label_loss = classifier_loss(source_label, source_label_output)

  source_domain = domain_classifier(source_feature)
  target_domain = domain_classifier(target_feature)
  domain_loss = d_loss(source_domain, target_domain)

  lamb = 0.1
  return source_label_loss - lamb * domain_loss

In [ ]:
epochs = 20

for i in range(epochs):
  print("Current epoch is:"+str(i))
  for j in range(20):
    x_test = test_data[5000*i:5000*(i+1), :, :, :]
    x_test = tf.reshape(x_test, (5000, 28, 28, 1))
    with tf.GradientTape() as e_tape, tf.GradientTape() as d_tape, tf.GradientTape() as c_tape:
      source_feature = extractor(train_data, training=True)
      target_feature = extractor(x_test, training=True)

      source_label = train_label
      source_label_output = label_classifier(source_feature)

      source_domain = domain_classifier(source_feature)
      target_domain = domain_classifier(target_feature)

      dd_loss = d_loss(source_domain, target_domain)
      ee_loss = extractor_loss(source_feature, target_feature, source_label)
      cc_loss = classifier_loss(source_label, source_label_output)

    gradient_of_d = d_tape.gradient(dd_loss, domain_classifier.trainable_variables)
    gradient_of_e = e_tape.gradient(ee_loss, extractor.trainable_variables)
    gradient_of_c = c_tape.gradient(cc_loss, label_classifier.trainable_variables)
  
    extractor_optimizer.apply_gradients(zip(gradient_of_e, extractor.trainable_variables))
    c_optimizer.apply_gradients(zip(gradient_of_c, label_classifier.trainable_variables))
    d_optimizer.apply_gradients(zip(gradient_of_d, domain_classifier.trainable_variables))

Current epoch is:0
